# Parallel version of the Eikonal code

In [3]:
import numpy as np

np.int(np.round(32/20))


2

In [1]:
## INITIALISE VARIABLES
import numpy as np
import ctypes
import multiprocessing as mp
import pymp
import time
import datetime
import math
import bisect
from os.path import isfile
import copy
import os
import psutil

def sortSecond(val):
    return val[1]
    
# Function to insert element 
def insertSorted(aList, newV):
    ini_index = 0
    end_index = len(aList)
    index = int((end_index-ini_index)/2)
    for i in range(0, len(aList), 1):
        if newV[1] < aList[index][1]:
            if end_index-ini_index <= 1 or index + 1 == end_index:
                index = index + 1
                break
            else:
                ini_index = index + 1
                index = int(index + (end_index-ini_index)/2 + 1)
        elif newV[1] > aList[index][1]:
            if end_index-ini_index <= 1 or index == ini_index:
                index = index # Place before the current position
                break
            else:
                end_index = index
                index = int(index - (end_index-ini_index)/2)
        else:
            index = ini_index
            break
    aList.insert(index, newV)

# Mesh name
meshSelected = 2
if meshSelected == 0:
    fileName = 'DTI003' # Mesh 0
elif meshSelected == 1:
    fileName = 'eikonal06' # Mesh 1
else:
    fileName = 'eikonal09' # Mesh 1

# Fibrosis
fibrosisFactors = [0]
no_rv_purkinje=0
perturb_roots=[1]
# Mesh detail
meshIsFine = 1
if meshIsFine:
    meshName = 'fine'
else:
    meshName = 'coarse'
# Endocardial Speed
endoSpeeds = [0.17,0.068]
# Epicardial Speed
epiSpeeds = [2]
comparison=0

study='root_calibration' 

#Duration 9798.086715221405
#endoSpeeds = [0.12]
#epiSpeeds = [1]
scaling=[1]
#Error 1.3772517598380738
z=0
a=np.shape(endoSpeeds)
b=np.shape(fibrosisFactors)
c=np.shape(scaling)
d=np.shape(perturb_roots)
tot=a[0]*b[0]*c[0]*d[0]
parameters=np.zeros((tot,8))


for l in range(0,a[0]):
    for j in range(0,b[0]):
        for k in range(0,c[0]):
            for o in range(0,d[0]):

                parameters[z][:] = np.array([ 0.0478*scaling[k],  0.0315*scaling[k], 0.0297*scaling[k], scaling[k] , fibrosisFactors[j] ,endoSpeeds[l],perturb_roots[o], no_rv_purkinje])
                z=z+1
#Duration 9822.775927066803
#endoSpeeds = [0.12]
#epiSpeeds = [2]
#configurations = [{'gl': 0.0667, 'gt': 0.0478, 'gn': 0.0443}] #Experiment 120 2x
#Error 1.5705925923403015

#Duration 10490.35331749916
#endoSpeeds = [0.179]
#epiSpeeds = [1]
#configurations = [{'gl': 0.0452, 'gt': 0.0315, 'gn': 0.0297}] #Experiment 179 1x
#Error 1.077697564564387

#Duration 10397.594599246979
#endoSpeeds = [0.179]
#epiSpeeds = [2]
#configurations = [{'gl': 0.0761, 'gt': 0.0469, 'gn': 0.0452}] #Experiment 179 2x
#Error 1.0097890059174461

print('Nb Confs ' + str(np.size(parameters,0)))

# Paths and tags
dataPath = 'metaData/' + fileName + '/'
dataPath1='metaData/' + fileName + '/'+study+'/'

try:
    # Create target Directory
    os.mkdir(dataPath1 )
    print("Directory " , (dataPath1 ),  " Created ") 
except FileExistsError:
        print("Directory " , (dataPath1  ),  " already exists")
        
print(parameters)

Nb Confs 2
('Directory ', 'metaData/eikonal09/root_calibration/', ' Created ')
[[0.0478 0.0315 0.0297 1.     0.     0.17   1.     0.    ]
 [0.0478 0.0315 0.0297 1.     0.     0.068  1.     0.    ]]


In [ ]:
## LOAD ALL THE DATA
nodesXYZ = np.loadtxt(dataPath + fileName + '_' + meshName 
                      + '_xyz.csv', delimiter=',')
edges = (np.loadtxt(dataPath + fileName + '_' + meshName 
                    + '_edges.csv', delimiter=',') - 1).astype(int)
lvface = np.unique((np.loadtxt(dataPath + fileName + '_' + meshName 
                               + '_lvface.csv', 
                               delimiter=',') - 1).astype(int)) # lv endocardium triangles
rvface = np.unique((np.loadtxt(dataPath + fileName + '_' + meshName 
                               + '_rvface.csv', 
                               delimiter=',') - 1).astype(int)) # rv endocardium triangles
tetraFibers = np.loadtxt(dataPath + fileName + '_' + meshName 
                         + '_tetrahedronFibers.csv', delimiter=',') # tetrahedron fiber directions
tetraFibers = np.reshape(tetraFibers, [tetraFibers.shape[0], 3, 3], order='F')
edgeVEC = nodesXYZ[edges[:, 0], :] - nodesXYZ[edges[:, 1], :] # edge vectors
if perturb_roots==1:
    rootNodesIndexes = (np.loadtxt(dataPath + fileName + '_' + meshName 
                               + '_rootNodes_fib.csv', 
                        delimiter=',') - 1).astype(int) # root nodes for the chosen mes
elif perturb_roots==2:
     rootNodesIndexes = (np.loadtxt(dataPath + fileName + '_' + meshName 
                               + '_rootNodes_fib2.csv', 
                        delimiter=',') - 1).astype(int) # root nodes for the chosen mesh
elif perturb_roots==3:
     rootNodesIndexes = (np.loadtxt(dataPath + fileName + '_' + meshName 
                               + '_rootNodes_fib3.csv', 
                        delimiter=',') - 1).astype(int) # root nodes for the chosen mesh
elif perturb_roots==4:
     rootNodesIndexes = (np.loadtxt(dataPath + fileName + '_' + meshName 
                               + '_rootNodes_fib4.csv', 
                        delimiter=',') - 1).astype(int) # root nodes for the chosen mesh
elif perturb_roots==5:
     rootNodesIndexes = (np.loadtxt(dataPath + fileName + '_' + meshName 
                               + '_rootNodes_fib5.csv', 
                        delimiter=',') - 1).astype(int) # root nodes for the chosen mesh
elif perturb_roots==6:
     rootNodesIndexes = (np.loadtxt(dataPath + fileName + '_' + meshName 
                               + '_rootNodes_fib6.csv', 
                        delimiter=',') - 1).astype(int) # root nodes for the chosen mesh
elif perturb_roots==7:
     rootNodesIndexes = (np.loadtxt(dataPath + fileName + '_' + meshName 
                               + '_rootNodes_fib7.csv', 
                        delimiter=',') - 1).astype(int) # root nodes for the chosen mesh
elif perturb_roots==9:
     rootNodesIndexes = (np.loadtxt(dataPath + fileName + '_' + meshName 
                               + '_rootNodes_rv_ant_middle.csv', 
                        delimiter=',') - 1).astype(int) # root nodes for the chosen mesh    
elif perturb_roots==10:
     rootNodesIndexes = (np.loadtxt(dataPath + fileName + '_' + meshName 
                               + '_rootNodes_no_septal_LV_post.csv', 
                        delimiter=',') - 1).astype(int) # root nodes for the chosen mesh         
elif perturb_roots==11:
     rootNodesIndexes = (np.loadtxt(dataPath + fileName + '_' + meshName 
                               + '_rootNodes_RV_posterior_moved.csv', 
                        delimiter=',') - 1).astype(int) # root nodes for the chosen mesh    
else:
     rootNodesIndexes = (np.loadtxt(dataPath + fileName + '_' + meshName 
                               + '_rootNodes.csv', 
                        delimiter=',') - 1).astype(int) # root nodes for the chosen mesh
    
numConf = np.size(parameters,1)
numEd = len(edges)
numNod = nodesXYZ.shape[0]

if meshIsFine==0:
    threadsNum = 32
else:
    threadsNum = 64
print('threadsNum ' + str(threadsNum))

# Set endocardial edges aside
if no_rv_purkinje==1:
    isEndocardial=np.bool_(np.all(np.isin(edges, lvface),axis=1))
else:
    isEndocardial=np.logical_or(np.all(np.isin(edges, lvface), axis=1), 
                      np.all(np.isin(edges, rvface), axis=1))
# Build adjacentcies
unfoldedEdges = np.concatenate((edges, np.flip(edges, axis=1))).astype(int)    
#edges = None # Clear Memory

aux = [[] for i in range(0, numNod, 1)]
for i in range(0, len(unfoldedEdges), 1):
    aux[unfoldedEdges[i, 0]].append(i)
neighbours = [np.array(n) for n in aux]
aux = None # Clear Memory


In [27]:
dataPath + fileName + '_' + meshName  + '_rootNodes_RV_anterior_'+np.str(np.int(parameters[sim_nr,6]))+'.csv'

'metaData/eikonal09/eikonal09_fine_rootNodes_RV_anterior_1.csv'

In [8]:
#########################np.concatenate(###############
# PARALLEL PART FOR EACH CONFIGURATION #
########################################
#start_time = time.time()

# Mem control
#process = psutil.Process(os.getpid())
#actMem = process.memory_info().rss*1e-9
#print('Ini actMem ' + str(actMem) + '   ' + str(psutil.virtual_memory().used*1e-9))

# Log file
f= open("eikonalLog.txt","a")
print("\nDate " + str(datetime.datetime.now()) + "\n" + meshName
        + "\n==========================\n==========================\n")
f.write("\nDate " + str(datetime.datetime.now()) + "\n" + meshName
        + "\n==========================\n==========================\n") # Log
f.close()

# Compute activations for an Endocardial speed
start_time = time.time()

activationResults = pymp.shared.array((numConf, nodesXYZ.shape[0]), dtype='uint32')#'uint32'
#activationResults = np.zeros((numConf, nodesXYZ.shape[0]), dtype='float64')
#if True:


if np.size(parameters,0)<40:
    nr_parallel_loops=1
elif np.size(parameters,0)<71:
    nr_parallel_loops=2
elif np.size(parameters,0)<100:
    nr_parallel_loops=3
else:
    nr_parallel_loops=5
    
    
batch=np.int(np.size(parameters,0)/nr_parallel_loops)
full=np.int(np.size(parameters,0))
for k in range(1,nr_parallel_loops+1):
    with pymp.Parallel(threadsNum) as p:
        for sim_nr in p.range(batch*(k-1), batch*k):
            fibrosisFactor=np.int(parameters[sim_nr,4])
            rootNodesIndexes = (np.loadtxt(dataPath + fileName + '_' + meshName  + '_rootNodes_RV_anterior_'+np.str(np.int(parameters[sim_nr,6]))+'.csv', delimiter=',') - 1).astype(int) # root nodes for the chosen mesh    

            if fibrosisFactor == 1:
                fibrosisTag = '1f'
                nodeFibrosisFactor = np.loadtxt(dataPath + fileName + '_' + meshName + '_nodeScaling_' + fibrosisTag + '.csv', delimiter=',')
                fibrosisDelay = np.mean(nodeFibrosisFactor[edges], axis=1) # numpy implemented arithmetic mean

            elif fibrosisFactor == 2:
                fibrosisTag = '2f'
                nodeFibrosisFactor = np.loadtxt(dataPath + fileName + '_' + meshName + '_nodeScaling_' + fibrosisTag + '.csv', delimiter=',')
                fibrosisDelay = np.mean(nodeFibrosisFactor[edges], axis=1) # numpy implemented arithmetic mean

            elif fibrosisFactor == 3:
                fibrosisTag = '3f'
                nodeFibrosisFactor = np.loadtxt(dataPath + fileName + '_' + meshName + '_nodeScaling_' + fibrosisTag + '.csv', delimiter=',')                           
                fibrosisDelay = np.mean(nodeFibrosisFactor[edges], axis=1) # numpy implemented arithmetic mean
            elif fibrosisFactor == 4:
                fibrosisTag = '1f_septal_3f'
                nodeFibrosisFactor = np.loadtxt(dataPath + fileName + '_' + meshName + '_nodeScaling_' + fibrosisTag + '.csv', delimiter=',')                           
                fibrosisDelay = np.mean(nodeFibrosisFactor[edges], axis=1) # numpy implemented arithmetic mean
            elif fibrosisFactor == 5:
                fibrosisTag = '2f_septal_4f'
                nodeFibrosisFactor = np.loadtxt(dataPath + fileName + '_' + meshName + '_nodeScaling_' + fibrosisTag + '.csv', delimiter=',')                           
                fibrosisDelay = np.mean(nodeFibrosisFactor[edges], axis=1) # numpy implemented arithmetic mean
            elif fibrosisFactor == 6:
                fibrosisTag = '3f_septal_5f'
                nodeFibrosisFactor = np.loadtxt(dataPath + fileName + '_' + meshName + '_nodeScaling_' + fibrosisTag + '.csv', delimiter=',')                           
                fibrosisDelay = np.mean(nodeFibrosisFactor[edges], axis=1) # numpy implemented arithmetic mean
            elif fibrosisFactor == 7:
                fibrosisTag = '1f_septal_whole_3f'
                nodeFibrosisFactor = np.loadtxt(dataPath + fileName + '_' + meshName + '_nodeScaling_' + fibrosisTag + '.csv', delimiter=',')                           
                fibrosisDelay = np.mean(nodeFibrosisFactor[edges], axis=1) # numpy implemented arithmetic mean
            elif fibrosisFactor == 8:
                fibrosisTag = '3f_septal_2f'
                nodeFibrosisFactor = np.loadtxt(dataPath + fileName + '_' + meshName + '_nodeScaling_' + fibrosisTag + '.csv', delimiter=',')                           
                fibrosisDelay = np.mean(nodeFibrosisFactor[edges], axis=1) # numpy implemented arithmetic mean
            elif fibrosisFactor == 9:
                fibrosisTag = '3f_septal_3f'
                nodeFibrosisFactor = np.loadtxt(dataPath + fileName + '_' + meshName + '_nodeScaling_' + fibrosisTag + '.csv', delimiter=',')                           
                fibrosisDelay = np.mean(nodeFibrosisFactor[edges], axis=1) # numpy implemented arithmetic mean
            elif fibrosisFactor == 10:
                fibrosisTag = '_apical_septal_2f'
                nodeFibrosisFactor = np.loadtxt(dataPath + fileName + '_' + meshName + '_nodeScaling_' + fibrosisTag + '.csv', delimiter=',')                           
                fibrosisDelay = np.mean(nodeFibrosisFactor[edges], axis=1) # numpy implemented arithmetic mean
            else:
                fibrosisTag = '0f'
                fibrosisDelay = np.ones((numEd,))           

            nodeFibrosisFactor = None # Clear memory
            endoSpeedTag = str(parameters[sim_nr][5]*100)+'x'

            scal=np.str(parameters[sim_nr][3])+'fiber'
            confTag = endoSpeedTag  + '_' + fibrosisTag+"_"+scal+"_roots_"+np.str(parameters[sim_nr,6])+"_no_rv_pur_"+np.str(no_rv_purkinje)
            try:
                # Create target Directory
                os.mkdir(dataPath1 + confTag )
                print("Directory " , (dataPath1 + confTag ),  " Created ") 
            except FileExistsError:
                print("Directory " , (dataPath1 + confTag ),  " already exists")

            # Precompute the cost of all endocardial edges
            endoSpeed=parameters[sim_nr][5]
            navigationCosts = np.zeros((numEd,))
            for index in range(0, numEd):
                if isEndocardial[index]:
                    # Cost for the propagation in the endocardium
                    navigationCosts[index] = math.sqrt(np.dot(edgeVEC[index, :], 
                                                              edgeVEC[index, :]))/endoSpeed

            # Current Configuration
            g = np.zeros((3, 3), float) # g matrix
            np.fill_diagonal(g, [parameters[sim_nr][0]**2, 
                                 parameters[sim_nr][1]**2, 
                                 parameters[sim_nr][2]**2],  wrap=False)# Needs to square each value


            for index in range(0, numEd):
                if not isEndocardial[index]:
                    # Cost equation for the Eikonal model + Fibrosis at the end
                    navigationCosts[index] = math.sqrt(
                        np.dot(
                            np.dot(
                                edgeVEC[index, :], 
                                np.linalg.inv(np.dot(tetraFibers[index, :, :], np.dot(g,
                                              tetraFibers[index, :, :].T)))
                            ),
                            edgeVEC[index, :][np.newaxis].T
                        )
                    )

            # Apply fibrosis
            navigationCosts = navigationCosts*fibrosisDelay

            # Build adjacentcy costs
            adjacentCost = [np.concatenate((unfoldedEdges[
                neighbours[i]][:, 1][:,np.newaxis], navigationCosts[
                neighbours[i]%numEd][:, np.newaxis]
                                            ), axis=1) for i in range(0, numNod, 1)]
            #delayedNavigationCosts = None # Clear Mem

            # COMPUTE ACTIVATION TIMES
            ## Initialise variables
            activationTimes = np.zeros((nodesXYZ.shape[0],), float) 
            visitedNodes = np.zeros((nodesXYZ.shape[0],), dtype=bool)
            cummCost = 1. # WARNING!! ROOT NODES HAVE A TIME OF 1 ms
            tempTimes = np.zeros((nodesXYZ.shape[0],), float) + 1000    


            ## Run the code for the root nodes
            visitedNodes[rootNodesIndexes] = True
            activationTimes[rootNodesIndexes] = cummCost
            #######################################
            # TEST USING BIDOMAIN ENDOCARDIAL ATM #
            #######################################
            #visitedNodes[rootNodesIndexes] = True
            #activationTimes[rootNodesIndexes] = bidomainATMap[rootNodesIndexes]
            #nextNodes = (np.vstack([adjacentCost[rootNodesIndex] 
            #                        + np.array([0, activationTimes[rootNodesIndex]])
            #                       for rootNodesIndex in rootNodesIndexes])).tolist()
            #######################################
            # TEST USING BIDOMAIN ENDOCARDIAL ATM #
            #######################################

            nextNodes = (np.vstack([adjacentCost[rootNodesIndex] + np.array([0, cummCost])
                                   for rootNodesIndex in rootNodesIndexes])).tolist()


            activeNode_i = rootNodesIndexes[0]
            nextNodes.sort(key=sortSecond, reverse=True)
            while visitedNodes[activeNode_i]:
                nextEdge = nextNodes.pop()
                activeNode_i = int(nextEdge[0])
            cummCost = nextEdge[1]
            tempTimes[(np.array(nextNodes)[:, 0]).astype(int)] = np.array(nextNodes)[:, 1]

            #p.print('2 actMem ' + str(process.memory_info().rss*1e-9) + '   ' 
            #        + str(psutil.virtual_memory().used*1e-9))

            ## Run the whole algorithm
            revise_count = 0
            for i in range(0, len(activationTimes)-(len(rootNodesIndexes)), 1):
                if  revise_count > (len(activationTimes)-len(rootNodesIndexes)
                                   )/128:# 80 was selected by trial and error comparing 
                    #execution times, later was increased to 128 to reduce mem usage 
                    #when 64 threads are called
                    revise_count = 0
                    tempVisited = copy.deepcopy(visitedNodes)
                    for j in reversed(range(0, len(nextNodes), 1)):
                        if tempVisited[int(nextNodes[j][0])]:
                            del nextNodes[j]
                        else:
                            tempVisited[int(nextNodes[j][0])] = True
                    tempVisited = None # Clear Mem
                visitedNodes[activeNode_i] = True
                activationTimes[activeNode_i] = cummCost
                adjacents = (adjacentCost[activeNode_i] + np.array([0, cummCost])).tolist()
                for adjacent_i in range(0, len(adjacents), 1):
                    if (not visitedNodes[int(adjacents[adjacent_i][0])] 
                    and (tempTimes[int(adjacents[adjacent_i][0])] > 
                    adjacents[adjacent_i][1])):
                        insertSorted(nextNodes, adjacents[adjacent_i])
                        tempTimes[int(adjacents[adjacent_i][0])] = adjacents[adjacent_i][1]
                        revise_count = revise_count + 1
                while visitedNodes[activeNode_i] and len(nextNodes) > 0:
                    nextEdge = nextNodes.pop()
                    activeNode_i = int(nextEdge[0])
                cummCost = nextEdge[1]  

            #p.print('3 actMem ' + str(process.memory_info().rss*1e-9) + '   ' 
            #        + str(psutil.virtual_memory().used*1e-9))

            # Clean Memory
            adjacentCost = None # Clear Mem
            visitedNodes = None # Clear Mem
            tempTimes = None # Clear Mem
            nextNodes = None # Clear Mem
            tempVisited = None # Clear Mem

            ## Save results
            #if np.amin(activationTimes) < 0.:
            #    p.print('Super BAD')
            activationResults[0, :] = (activationTimes+.5).astype('uint32', casting='unsafe')
            #activationResults[confIndex, :] = activationTimes
            activationTimes = None # Clear Mem

            #p.print('4 actMem ' + str(process.memory_info().rss*1e-9) + '   ' 
            #        + str(psutil.virtual_memory().used*1e-9))




            bestIndex=0;
            np.savetxt(dataPath1  + confTag+'/' + confTag
                   + '_NEW_pred_ATMap' + '.csv', 
                   activationResults[bestIndex, :], delimiter=',')
            bestError=0



            # Mem control
            #actMem = process.memory_info().rss*1e-9
            #print(str(confIndex) + '\n11 actMem-antMem ' + str(actMem-antMem) 
            #              + '\nactMem ' + str(actMem))
            #antMem = actMem 


            print('Experiment ' + confTag
                 + '\nError: ' + str(bestError)
                  + '\nBest Conf: ' + str(bestIndex) + '\n' 
                 + '\n'  + '------------------------------\n')


   


Date 2019-11-25 15:32:37.554405
fine


Date 2019-11-25 15:32:37.554405
fine

Directory  metaData/eikonal09/exp8/6.800000000000001x_0f_1.0fiber_roots_1.0_no_rv_pur_0  Created 
Directory  metaData/eikonal09/exp8/17.0x_0f_1.0fiber_roots_1.0_no_rv_pur_0  already exists
Experiment 6.800000000000001x_0f_1.0fiber_roots_1.0_no_rv_pur_0
Error: 0
Best Conf: 0

------------------------------

Experiment 17.0x_0f_1.0fiber_roots_1.0_no_rv_pur_0
Error: 0
Best Conf: 0

------------------------------

